In [3]:
import pickle
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
X,y = pickle.load(open('../data/xyhighpass.p','rb'))

In [4]:
X = np.nan_to_num(X[:,:,:3])
y = np.array(y)
groups = np.array(np.argmax(y,axis=1)).reshape(-1)
def separate_known_unknown_users(X,y,groups,split_ratio = .2):
    from sklearn.model_selection import LeavePGroupsOut
    n_users = len(np.unique(groups))
    lpgo = LeavePGroupsOut(n_groups=int(n_users*split_ratio))
    for train_index, test_index in lpgo.split(X, y, groups=groups):
        X_keep, X_out = X[train_index], X[test_index]
        y_keep, y_out = y[train_index], y[test_index]
        groups_keep, groups_out = groups[train_index], groups[test_index]
        break
    return X_keep,X_out,y_keep,y_out,groups_keep,groups_out
X_keep,X_test_out,y_keep,y_test_out,groups_keep,groups_test_out = separate_known_unknown_users(X,y,groups,split_ratio=.2)
X_in,X_val_out,y_in,y_val_out,groups_in,groups_val_out = separate_known_unknown_users(X_keep,y_keep,groups_keep,split_ratio=.1)
X_train, X_test_in, y_train, y_test_in,groups_train,groups_test_in = train_test_split(X_in, y_in,groups_in, test_size=0.2, random_state=42,stratify=groups_in)
X_train_in, X_val_in, y_train_in, y_val_in,groups_train_in,groups_val_in = train_test_split(X_train, y_train,groups_train, test_size=0.1, random_state=42,stratify=groups_train)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder, MNIST
from torchvision import transforms,datasets
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
from torchsummary import summary
import torch.utils.data as data_utils
batch_size = 256
train = data_utils.TensorDataset(torch.from_numpy(X_train_in), torch.from_numpy(y_train_in),torch.from_numpy(groups_train_in))
val = data_utils.TensorDataset(torch.from_numpy(X_val_in), torch.from_numpy(y_val_in),torch.from_numpy(groups_val_in))
test = data_utils.TensorDataset(torch.from_numpy(X_test_in), torch.from_numpy(y_test_in),torch.from_numpy(groups_test_in))
data_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

In [ ]:
class Generator(nn.Module): 
    def __init__(self):
        super(Generator, self).__init__()
        self.n_classes = 5
        self.std = .002
        self.output_length = 256*6
        self.noise_length = 100
        self.intermediate_length = 800
        self.n_channels = 6
        self.n_timesteps = 256
        self.model_noise = nn.Sequential(
            nn.Linear(self.noise_length,self.n_timesteps*self.noise_length//10),
            nn.Tanh()
        )
        self.model_noise2 = nn.Sequential(
            nn.Conv1d(10,self.n_timesteps,kernel_size=10),
            nn.Tanh()
        )
        
        self.model_label = nn.Sequential(
            nn.Linear(self.n_classes,self.n_timesteps),
            nn.ReLU()
        )
        self.model_label2 = nn.Sequential(
            nn.Conv1d(1,self.n_timesteps,kernel_size=10),
            nn.ReLU()
        )
        
        self.model_final = nn.Sequential(
            nn.Conv1d(494,100,kernel_size=10),
            nn.Tanh(),
            nn.Conv1d(100,100,kernel_size=5),
            nn.Tanh(),
            nn.Conv1d(100,6,kernel_size=4,padding=8),
            nn.Tanh()
            
            
        )